# Problem Set 1 - Working with Datasets and Exploring Pandas and Plots

__TA__: _Madhumitha Saksthi_ <br>
__Instructor__: _Ramesh Yerraballi_ <br>
__Semester__: _Fall 2019_

This problem set consist of 5 problems in total. The first two problems would give you an insight on using pandas. The next problem will focus on cleaning the dataset for missing datapoints. Finally, the last two problems would involve generating graphs based on the traffic dataset. 

This exercise discovers an extensive dataset on the subject of traffic on German roads provided by the BASt. It holds detailed numbers of cars, trucks and other vehicle groups passing more than 1,500 automatic counting stations. This dataset records for each counting station are provided on an hourly basis and they reach back to the year 2003.

We will explore some new insights about the data given.

Note for checking the code: 
1. Please be sure to provide relative path in all places where you are loading data from a location.
2. Give clear spacing and comments for the solution code. 
3. If there is error running the code, points will be deducted. So, please submit a working code. 

# Import Libraries and Download data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# download file from http://www.bast.de/videos/2016_A_S.zip
df = pd.read_csv(r'2016_A_S.txt', sep=';')

colors = ['#2678B2', '#AFC8E7', '#FD7F28', '#FDBB7D', 
          '#339E34', '#9ADE8D', '#D42A2F', '#FD9898', 
          '#9369BB', '#C5B1D4', '#8B564C', '#C39C95', 
          '#E179C1', '#F6B7D2', '#7F7F7F', '#C7C7C7']

In [ ]:
#extract required columns and translate column names to English
df = df[['Zst', 'Land', 'KFZ_R1', 'KFZ_R2', 'Datum', 'Stunde']]
df.columns = ['nr', 'land', 'cars_r1', 'cars_r2', 'date', 'hour']
df

## Pandas: exercise 1
In this exercise, print the data frame and you will notice that the format in which date has been provided as 161231 for 2016-12-31. Write a one liner code to convert the 'date' to DateTime format.The resulting date should appear as '2016-01-01 01:00:00' based on your code.  

In [ ]:
# convert values in 'date' column to DateTime
df.loc[df.hour == 24, 'hour'] = 0
# Typecast date and hour columns to string & concatenate to represent "yymmdd-hh" format; Use to_datetime fucntion 
# to convert string to required timestamp 
df['date'] =  pd.to_datetime( df['date'].astype(str) + '-' + df['hour'].astype(str), format='%y%m%d-%H')
df.date

## Pandas: exercise 2
In this exercise, based on the total sum across time for each land, print the land parameter with the maximum traffic. You would be using boolean indexing to indetify the land with the maximum traffic.Typically, you should be identifying the land with maximum traffic as one among the 16 values. 

In [ ]:
df_sum = df
df_sum['sum'] = df_sum['cars_r1'] + df_sum['cars_r2']
# Step 1: Calculate total traffic for each land across all dates; achieved by grouping df_sum with yearly freq
df_byday_sum = df_sum.set_index('date').groupby(['land', pd.Grouper(freq='Y')])[['sum']].sum().reset_index()
# print(df_byday_sum)
# Identify land with maximum total traffic (cars1 + cars 2) by locating dataframe value which is equal to max value
# of dataframe
print("Land with the maximum traffic is:", df_byday_sum.loc[df_byday_sum['sum'] == df_byday_sum['sum'].max(), 'land'].values[0])

## Missing: exercise 3
The code below will randomly generate index and remove the data point from those index. Using a technique of filling missing data, we will try to recover the lost data in the following exercise. 

In [ ]:
# Generate random indexes 
import random
import numpy  as np
index = []
for x in range(1000):
    index.append(random.randint(1,7273152))
index = np.unique(index)
df_missing = df
df_missing.loc[index,'cars_r1'] = np.nan
df_missing.loc[index,'cars_r2'] = np.nan

In [ ]:
#When you print the dataframe you will notice that the values are set to NaN. 
print(df_missing.loc[index])

# 3.1
In what conditions should NaN values be filled with mean and median? 
(Hint: Consider outliers in the data)

In [ ]:
#3.1(answer): Replacing NaN with mean is suitable for continuous data without outliers 
#             Replacing NaN with median is suitable for continuous data with outliers


#3.2 Write the code to drop the rows with missing values 
df_part2 = df_missing.dropna(subset=['cars_r1', 'cars_r2'])
#print(df_part2)

#3.3 Fill the value with the previous time stamp value
df_part3 = df_missing.fillna(method='ffill')
#print(df_part3.loc[index])


#3.4 Fill the value with a constant 
df_part4 = df_missing.fillna(0)
#print(df_part4.loc[index])


#3.5 Fill the value with the mean of value above and below the datapoint 
df_part5 = df_missing
df_part5['cars_r1'] = (df_missing['cars_r1'].fillna(method='ffill') + df_missing['cars_r1'].fillna(method='bfill'))/2
df_part5['cars_r2'] = (df_missing['cars_r2'].fillna(method='ffill') + df_missing['cars_r2'].fillna(method='bfill'))/2
print(df_part5.loc[index])

## Plot: exercise 4

# Plot linechart of number of cars_r1 and cars_r2 vs date with different colors and legend.

Using the `station` and `station_days`. Analyze what station and station_days represent and how they should be used. It is pretty straight forward.
The plots have been generated and given below for your reference. The code writtern should generate similar graphs. 

In [ ]:
grouped_by_name = df.groupby(['nr'])
number = 1177
station = grouped_by_name.get_group(number)
station_days = station.set_index('date').groupby([pd.Grouper(freq='D')])[['cars_r1', 'cars_r2']].sum().reset_index()
#print(station)
print(station_days)

In [ ]:
# Set width, height, resolution of figure
fig, ax = plt.subplots(figsize=(15, 8), dpi=200)
# Plot sublot1 (x,y) for cars_r1 in blue color
ax.plot(station_days['date'], station_days['cars_r1'], c=colors[0])
# Plot subplot2 (x,y) for cars_r2 in orange color
ax.plot(station_days['date'], station_days['cars_r2'], c=colors[2])
# Format Y-Axis labels as multiples of thousand
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
# Place legend outside the plot in top right position
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.015))
# Set title for the plot 
plt.title('station nr: ' + str(number))
plt.margins(0)
plt.show()

## Plot exercise 5

## Scatterplot: Top 3 busiest days per state

In [ ]:
df_scatter = df
df_scatter['sum'] = df_scatter['cars_r1'] + df_scatter['cars_r2']
df_byday = df_scatter.set_index('date').groupby(['land', pd.Grouper(freq='D')])[['sum']].sum().reset_index()
df_sorted = df_byday.sort_values(by=['sum'], ascending=False)
df_clean = df_sorted.groupby(['land']).head(3)
df_clean

# Plot the different group of land vs date and show the 3 busiest days using the above DataFrames generated.

You have to just write one line inside the for loop and plot.

In [ ]:
groups = df_clean.groupby('land')
scatter_colors = iter(colors)

fig, ax = plt.subplots(figsize=(15, 8), dpi=200)
for name, group in groups:
    dates = group['date'].dt.strftime('%Y-%m-%d').values
    dates = [pd.to_datetime(d) for d in dates]
    #Use scatter function to create scatterplots; args: x,y,area,color,label
    ax.scatter(dates, group['sum'].values, s=200, c=next(scatter_colors), label=name)

ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.015))
plt.show()